In [8]:
%load_ext autoreload
%autoreload 2

import os, sys
# Set absolute path to the root folder of the directory
full_path = os.getcwd()
home_folder = 'CPS_GradRate_Analysis'
root = full_path.split(home_folder)[0] + home_folder + '/'
sys.path.append(root)

import pandas as pd
import numpy as np

from src.cleaning import prep_high_school_dataframe
from src.cleaning import filter_cwoption_special_ed
from src.cleaning import delta_student_count

from src.cleaning import import_and_merge_data, drop_no_students, drop_no_grad_rate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
path_to_pr_1819 = root + 'data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Progress_Reports_SY1819.csv'
path_to_sp_1819 = root + 'data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv'

df_1819 = import_and_merge_data(path_to_sp_1819, path_to_pr_1819)

# Drop Schools with no recorded students

Some of the school records have 0 students recorded.  These are dropped by the cleaning function.

In [40]:
df_1819 = drop_no_students(df_1819)

0 Student Count
2 schools
15     ENGLEWOOD STEM HS
582       YCCS - VIRTUAL
Name: Short_Name_sp, dtype: object
All 0 Student Count Schools Dropped


# Drop Schools with no graduation rate recorded

In [41]:
df_1819 = drop_no_grad_rate(df_1819)

0 Graduation Rate
2 schools
261    PATHWAYS - AVONDALE HS
343     NORTHSIDE LEARNING HS
Name: Short_Name_sp, dtype: object
All 0 Graduation Rate Schools Dropped


## Change from Prior Year

Changes in the student populations across years are interesting reflections of school performance. In order to add this data to the dataframes, certain years from the prior year's school profile must be added.  The first column to be added is the change in Total_Student_Count.  Once change in student count has been added, I will look at the correlation to graduation rate.  

In [18]:
from src.cleaning import import_and_merge_data

path_to_prior_year_sp = root + 'data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1718.csv'
path_to_prior_year_pr = root + 'data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Progress_Reports_SY1718.csv'

df = delta_student_count(df_1819, path_to_previous_year_sp, path_to_previous_year_pr, new_year_added='1718')

In [20]:
df_high_school = prep_high_school_dataframe(path_to_sp_1819, path_to_pr_1819, 
                                            path_to_previous_year_sp, 
                                            path_to_prior_year_pr)

0 Student Count
2 schools
15     ENGLEWOOD STEM HS
582       YCCS - VIRTUAL
Name: Short_Name_sp, dtype: object
All 0 Student Count Schools Dropped
0 Graduation Rate
2 schools
261    PATHWAYS - AVONDALE HS
343     NORTHSIDE LEARNING HS
Name: Short_Name_sp, dtype: object
##########
NA Graduation Rates
38 schools
All 0/NA Graduation Rate Schools Dropped


In [23]:
df_high_school[['Graduation_Rate_School', 'student_count_total_change_1_year']].corr()

,Graduation_Rate_School,student_count_total_change_1_year
Graduation_Rate_School,1.000000,-0.176734
student_count_total_change_1_year,-0.176734,1.000000


In [25]:
df_high_school['Graduation_Rate_School'].isna().sum()

0

From the above feature engineering, we are left with a column describing change in student count from the prior year. It has a weak negative correlation to graduation rate of the school.

# Future Steps

- Instead of dropping No Student Counts, fill with last year's counts, or fill by adding up counts from other columns.